In [11]:
# Start off with imports
import spacy
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [13]:
# What is lemma?
# A lemma is a step past stemming in out NLP processessing. IT takes the meaning of the word
# First thing we have to do is create a dataframe object:
df = pd.read_csv(r'C:\Users\jmira\OneDrive\Coding Temple Cohort 0501 0623\github\bonfire119_mongodb_streamlit\oracle_cards.csv')
df

C:\Users\jmira\AppData\Local\Temp\ipykernel_22056\1806063485.py:4: DtypeWarning: Columns (74,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\jmira\OneDrive\Coding Temple Cohort 0501 0623\github\bonfire119_mongodb_streamlit\oracle_cards.csv')


,Unnamed: 0,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,...,frame_effects,tcgplayer_etched_id,promo_types,loyalty,life_modifier,hand_modifier,attraction_lights,color_indicator,content_warning,flavor_name
0,0,card,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,0004ebd0-dfd6-4276-b4a6-de0003e94237,[15862],15870.0,15871.0,3094.0,3081.0,Static Orb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,card,7050735c-b232-47a6-a342-01795bfd0d46,0006faf6-7a61-426c-9034-579f2cfcfa83,[370780],49283.0,49284.0,69965.0,262945.0,Sensory Deprivation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,card,e718b21b-46d1-4844-985c-52745657b1ac,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,[470580],77122.0,NaN,196536.0,391692.0,Road of Return,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,card,036ef8c9-72ac-46ce-af07-83b79d736538,000d5588-5a4c-434e-988d-396632ade42c,[83282],22609.0,22610.0,12835.0,12551.0,Storm Crow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,card,b125d1e7-5d9b-4997-88b0-71bdfc19c6f2,000e5d65-96c3-498b-bd01-72b1a1991850,[12380],12637.0,12638.0,6412.0,10604.0,Walking Sponge,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28901,28901,card,d6695fa8-881c-407c-91d9-3ac770372d35,fffa9334-3576-4f70-9605-2ad062cdbc69,[430770],64646.0,64647.0,136683.0,298834.0,Without Weakness,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28902,28902,card,4d5edf4f-5695-42fc-9e57-c4faef60fbc3,fffaa634-42a0-4038-b027-24f28754fec7,[571548],101914.0,NaN,276869.0,665173.0,Firesong and Sunspeaker,...,['legendary'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28903,28903,card,22a6a5f1-1405-4efb-af3e-e1f58d664e99,fffcb71f-e802-436e-917d-eaa6607ab74f,"[503766, 503767]",87643.0,NaN,230214.0,530502.0,"Toralf, God of Fury // Toralf's Hammer",...,['legendary'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28904,28904,card,09f06f55-7918-46c4-80ff-0bf39e091a4a,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,[430833],64772.0,64773.0,132234.0,298282.0,"Samut, the Tested",...,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Let's preiew the column we're trying to transform
df['oracle_text']

0        As long as Static Orb is untapped, players can...
1         Enchant creature\nEnchanted creature gets -3/-0.
2        Choose one —\n• Return target permanent card f...
3        Flying (This creature can't be blocked except ...
4        {T}: Target creature loses your choice of flyi...
                               ...                        
28901    Target creature you control gains indestructib...
28902    Red instant and sorcery spells you control hav...
28903                                                  NaN
28904    +1: Up to one target creature gains double str...
28905                      All Sliver creatures get +1/+1.
Name: oracle_text, Length: 28906, dtype: object

In [6]:
!python -m spacy download en_core_web_md

                                              0.0/42.8 MB ? eta -:--:--
                                              0.2/42.8 MB 4.1 MB/s eta 0:00:11
                                              0.6/42.8 MB 6.6 MB/s eta 0:00:07
     -                                        1.6/42.8 MB 11.4 MB/s eta 0:00:04
     --                                       2.5/42.8 MB 13.4 MB/s eta 0:00:04
     --                                       3.1/42.8 MB 13.0 MB/s eta 0:00:04
     ---                                      4.0/42.8 MB 14.3 MB/s eta 0:00:03
     ----                                     5.1/42.8 MB 15.6 MB/s eta 0:00:03
     -----                                    6.2/42.8 MB 16.6 MB/s eta 0:00:03
     ------                                   7.3/42.8 MB 17.4 MB/s eta 0:00:03
     --------                                 8.7/42.8 MB 18.6 MB/s eta 0:00:02
     ---------                               10.2/42.8 MB 19.7 MB/s eta 0:00:02
     ----------                              11.4/

In [15]:
# Dropping all null values form the oracle text column
df.dropna(subset=['oracle_text'], axis = 0, inplace=True)

# Dropping all values from the oracle text column that are empty
df.drop(df.index[df['oracle_text'] == ''], inplace=True)
df['oracle_text']

0        As long as Static Orb is untapped, players can...
1         Enchant creature\nEnchanted creature gets -3/-0.
2        Choose one —\n• Return target permanent card f...
3        Flying (This creature can't be blocked except ...
4        {T}: Target creature loses your choice of flyi...
                               ...                        
28899    When Quarry Beetle enters the battlefield, you...
28901    Target creature you control gains indestructib...
28902    Red instant and sorcery spells you control hav...
28904    +1: Up to one target creature gains double str...
28905                      All Sliver creatures get +1/+1.
Name: oracle_text, Length: 26604, dtype: object

In [16]:
df['oracle_text'] = [re.sub('[^0-9a-zA-Z]+', " ", i) for i in df.oracle_text]
# reggex is looking for everything NOT a alphanumeric item
df['oracle_text']

0        As long as Static Orb is untapped players can ...
1            Enchant creature Enchanted creature gets 3 0 
2        Choose one Return target permanent card from y...
3        Flying This creature can t be blocked except b...
4         T Target creature loses your choice of flying...
                               ...                        
28899    When Quarry Beetle enters the battlefield you ...
28901    Target creature you control gains indestructib...
28902    Red instant and sorcery spells you control hav...
28904     1 Up to one target creature gains double stri...
28905                        All Sliver creatures get 1 1 
Name: oracle_text, Length: 26604, dtype: object

In [18]:
#Using spacy. we need to instatiate the dictionary
nlp = spacy.load('en_core_web_md')
lemmas = []
for doc in df['oracle_text']:
    lemmas.append([token.lemma_.lower().strip() for token in nlp(str(doc)) if (token.is_stop != True) and (token.is_punct != True) and (token.is_space != True)])
df['lemmas'] = lemmas

In [20]:
# Create a data vectorizor to save our vocabulary and map out our lemmas to a euclidean plane

# to do so, we wil lcreate a function called dummy_fun that takes in a doc and returns it
def dummy_fun(doc):
    return doc

# Create vect:
vect = TfidfVectorizer(preprocessor=dummy_fun,
                       token_pattern=None,
                       tokenizer=dummy_fun)

# Fit the vectorizer
vect.fit(df.lemmas)


TfidfVectorizer(preprocessor=<function dummy_fun at 0x000001B8E7452680>,
                token_pattern=None,
                tokenizer=<function dummy_fun at 0x000001B8E7452680>)

In [25]:

# Last step for our modeling is to create a model and fit it
model = NearestNeighbors(n_neighbors=12)
model.fit(vect.transform(df.lemmas))

x, y = model.kneighbors(vect.transform(df['lemmas'][df['name'] == 'The World Tree']))
for val in y:
    display(df['name'][val])


5315           Morphic Tide
17483        Mardu Outrider
21701      Act of Authority
6556      Release to Memory
14638       Swift Reckoning
330          Jasconian Isle
16159      Arachnus Spinner
24369              Success!
20916    Crystalline Sliver
9542             Blood Lust
11548             Persecute
21812     Temporal Trespass
Name: name, dtype: object